## Use the Fred API

In [1]:
from full_fred.fred import Fred
import pandas as pd
fred = Fred('fred_key.txt')
fred.set_api_key_file('fred_key.txt')

True

In [2]:
fred.env_api_key_found()

False

In [3]:
## fetch the main data -- home price index
home_price = fred.get_series_df("CSUSHPINSA").drop(["realtime_start", "realtime_end"], axis = 1)
home_price = home_price[home_price["value"] != "."].reset_index(drop = True)
home_price

,date,value
0,1987-01-01,63.735
1,1987-02-01,64.135
2,1987-03-01,64.47
3,1987-04-01,64.973
4,1987-05-01,65.547
...,...,...
434,2023-03-01,297.318
435,2023-04-01,301.462
436,2023-05-01,305.421
437,2023-06-01,308.316


In [4]:
## save the data 
home_price.to_csv("../data/national_home_price_index.csv", index = False)

#### Choose some relevant indicators at FRED:

1. Economic increment: GDP, savings, Capacity Utilization

2. inflation: CPI, PPI, PCE

3. Interest Rate: fed fund rate, real rate, tips rate

4. Employment: unemployment rate, non-agricultural employment

5. fiscal policy: bond issue / monetary policy

6. Housing inventory 

7. House prices

【FRED】

Monthly:
* 10-Year Real Interest Rate（REAINTRATREARAT10Y)
* Capacity Utilization: Total Index (TCU)
* Sticky Price Consumer Price Index (STICKCPIM157SFRBATL)
*  Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUCSL)
* Personal Consumption Expenditures (PCE)
* Federal Funds Effective Rate (FEDFUNDS)
* Unemployment Rate (UNRATE)
* Employment Level - Nonagricultural Industries (LNS12035019) / Employment Level (CE16OV)
* New Privately-Owned Housing Units Under Construction: Total Units (UNDCONTSA)
* New Privately-Owned Housing Units Completed: Total Units (COMPUTSA)
* Personal Saving Rate (PSAVERT)
* New Privately-Owned Housing Units Started: Total Units (HOUST)
* Median Sales Price for New Houses Sold in the United States (MSPNHSUS)
* 30-Year Fixed Rate Mortgage Average in the United States (MORTGAGE30US)

Quarterly: 
* (GDP) 






#### Indicators documents: 

We need to assume that the monthly indicators listed above **release timely**, that is to say it can be used to predict the unreleased home price index with no look-ahead bias. Because the release of these timely indicators is usually delayed for only 1 month, while the home price index may be delayed longer, for about 1 quarter.  

need adjustment to ensure no look-ahead bias: (quarterly and release 1 month later)
* (GDP)

In [7]:
## get the data 
data_dic = {
    "real_interest_rate":"REAINTRATREARAT10Y",
    "capacity_util":"TCU",
    "stickyCPI":"STICKCPIM157SFRBATL",
    "CPI":"CPIAUCSL",
    "PCE":"PCE",
    "fed_fund_rate":"FEDFUNDS",
    "unemploy_rate":"UNRATE",
    "employ_level_nonagri":"LNS12035019",
    "employ_level":"CE16OV",
    "housing_started":"HOUST",
    "housing_completed":"COMPUTSA",
    "GDP":"GDP",
    "personal_saving_rate":"PSAVERT",
    "avg_house_prices":"MSPNHSUS",
    "mortgage":"MORTGAGE30US"
}

In [8]:
## fetch the data
def fetch_data(data_dic, address = "../data/"):
    for key in data_dic.keys():
        ticker = data_dic[key]
        data = fred.get_series_df(ticker).drop(["realtime_start", "realtime_end"], axis = 1)
        data = data[data["value"] != "."].reset_index(drop = True)
        data.to_csv(address + key + ".csv", index = False)
        print("Successfully load and save data: "+ ticker)

fetch_data(data_dic)

Successfully load and save data: REAINTRATREARAT10Y
Successfully load and save data: TCU
Successfully load and save data: STICKCPIM157SFRBATL
Successfully load and save data: CPIAUCSL
Successfully load and save data: PCE
Successfully load and save data: FEDFUNDS
Successfully load and save data: UNRATE
Successfully load and save data: LNS12035019
Successfully load and save data: CE16OV
Successfully load and save data: HOUST
Successfully load and save data: COMPUTSA
Successfully load and save data: GDP
Successfully load and save data: PSAVERT
Successfully load and save data: MSPNHSUS
Successfully load and save data: MORTGAGE30US


In [ ]:
### afjust the weekly data mortgage
ticker = "MORTGAGE30US"
data = fred.get_series_df(ticker).drop(["realtime_start", "realtime_end"], axis = 1)
data = data[data["value"] != "."].reset_index(drop = True)
data["date"] = data["date"].apply(lambda x: pd.to_datetime(x))
data["value"] = pd.to_numeric(data["value"])
data = data.set_index("date")
data = data.resample("1M").mean().reset_index()

In [34]:
data["date"] = data["date"].apply(lambda x: pd.to_datetime("%d-%.02d-01"%(x.year, x.month)))
data.to_csv("../data/mortgage"+ ".csv", index = False)

## Use Yahoo API 


In [9]:
from yahoo_fin.stock_info import get_data
import pandas as pd

In [10]:
start= "01/01/2000"
end = "10/19/2023"
tickers = ["goog", "amzn", "jpm", "gme", "xom", "spy"]
datafile = "../data/"

for tk in tickers:
    data = get_data(tk, start_date = start, end_date = end, index_as_date = False,
                interval = "1d") 
    data.to_csv(datafile + tk + ".csv", index = False)
    print("Successfully load and save data: " + tk)

Successfully load and save data: goog
Successfully load and save data: amzn
Successfully load and save data: jpm
Successfully load and save data: gme
Successfully load and save data: xom
Successfully load and save data: spy
